# <ins>Rolling Windows in Python: NumPy & scikit-image</ins>

Here we explore the idea of rolling windows and aggregate functions on them. This assumes the reader already has familiarity with (multidimensional) arrays and slices. It also assumes the reader is familiar with [NumPy](https://numpy.org) arrays and the semantics of addressing their shapes and dimensions.

Let's import numpy first:

In [1]:
import numpy as np

## <ins>Review of slices in numpy</ins>

Say we have an array of dimensions (7, 9), which we’ll call `arr`:

In [2]:
arr = np.arange(7*9).reshape(7, 9)
arr

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23, 24, 25, 26],
       [27, 28, 29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49, 50, 51, 52, 53],
       [54, 55, 56, 57, 58, 59, 60, 61, 62]])

A slice of size (2, 4) with starting corner at (1, 2) would look like this:

In [3]:
arr[1:1+2, 2:2+4]

array([[11, 12, 13, 14],
       [20, 21, 22, 23]])

Note, numpy allows us to take such slices even at indices where the slice doesn’t fit, silently truncating the result. Thus, if we tried taking the slice<br>`arr[5 : 5+2, 6 : 6+4]`, we would get a subarray of size (2, 3) rather than (2, 4):

In [4]:
arr[5 : 5+2, 6 : 6+4]

array([[51, 52, 53],
       [60, 61, 62]])

Given the starting index (5, 6), there are only three columns available from that point in the array to the array’s final column, so that is all we get in the slice.

In general, for an array `A` of `d` dimensions, a slice of size $$(s_0, s_1, …, s_{d-1} )$$ at index $$[i_0, i_1, …, i_{d-1}]$$ can be obtained by indexing into `A` with the slice<br>
$$A[i_0:i_0+s_0, i_0:i_0+s_0, …, i_{d-1}:i_{d-1}+s_{d-1}]$$

If `A` has dimensions $(a_0,a_1,…,a_{d-1})$ The last index where the slice fits is at $$A[\ \ a_0 - s_0,\ \  a_1 - s_1,\ \  …,\ \  a_{d-1} - s_{d-1}\ \ ]$$

In the above array, this means the last location that will accommodate a slice of size `(2, 4)` is `(7-2, 9-4)`, or `(5, 5)`. The slice starting at this location is<br><center><code>[[ 50, 51, 52, 53]<br>   [ 59, 60, 61, 62]]</code></center>

The most important thing about a numpy slice is that it is a *view* into the source array, rather than a new array entirely. This means that internally, a numpy array constructed from a slice of another array is simply a pointer into the host array, along with information about the shape of the slice. This is different from Python's list and tuple semantics, where taking a slice involves copying elements.

## <ins>Rolling windows</ins>

The concept of a rolling window is a natural extension of taking a slice. Given a slice of some size, a rolling window means taking that slice from different starting positions in the array, traversing the array in the same order we would normally traverse its elements.

To demonstrate, let’s take `arr` from above, again using a slice of size (2, 4). If we only wanted single elements, iterating over the array in row-major order, we begin at index [0, 0] and then iterate over the first row:

`[0, 1, 2, 3, 4, 5, 6, 7, 8]`

If we were to spell out the indices we were using, it would look like this:

`[[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7], [0,8]]`

The idea of a rolling window is to enumerate the indices in the same order, but instead of returning single elements at each index, return a slice at each index. Thus, for a rolling window (slice) of size (2, 4) we would take the following slices from the first row:

`[[0:2,0:4], [0:2,1:5], [0:2,2:6], ..., [0:2,5:9]]`

Notice, we stop at `[0:2,5:9]` because the highest range of 4 columns we can extract from this array is `(5,6,7,8)`, which we get from the column slice `5:9`.

The slices we would wind up with would be<br>
`[[ 0, 1, 2, 3]   [[ 1, 2, 3, 4]  …  [[ 5, 6, 7, 8],
  [ 9,10,11,12]]   [10,11,12,13]] …   [14,15,16,17]] `

Notice, we move forwards one column at a time, constantly maintaining a window size of `(2, 4)`. Between each iteration, the first column of the subarray is removed, and the next available column is appended after the subarray’s last column. This continues until our subarray overlaps the last column in `arr`, at which point we cannot iterate any further in the first row.

After exhausting the first row, we continue onwards through the second row in the same fashion:<br>
`[[ 9,10,11,12]]   [[10,11,12,13]]  …  [[14,15,16,17]]
  [18,19,20,21]]    [19,20,21,22]]  …   [20,21,22,23]]`

Notice, the first row in each subarray here was the second row in the corresponding subarray in the previous iteration. We have moved down one row, so our sliding windows taken at the second row (which have more than one row) overlap with those taken at the first row.

We could continue this process throughout the whole of `arr`; the last subarray we would encounter would be<br>
<code>[[ 50, 51, 52, 53]<br>  [ 59, 60, 61, 62]]</code>

Which overlaps with the last row and last column of `arr`.

## <ins>Properties of the rolling window</ins>

In order for the rolling window to be well-defined, it must have the same number of dimensions as its host array. It can, however, have size 1 along any of these dimensions, effectively making it a higher-dimensional view of a lower-dimensional slice. For example, instead of `(2, 4)`, say our slice size into `arr`was `(1, 4)`. Then starting at index `[0, 0]`, our first slice would have been<br><br>`[[0, 1, 2, 3]]`

Notice the double brackets: even though there is only one row in this subarray, it technically has two dimensions, as this is required for the rolling window to be well defined. However, we are certainly welcome to think of this as a one-dimensional slice; but we would have to manually reduce its dimension in code if we wanted numpy to interpret it as a one-dimensional array.

Another way of saying this is that a rolling window (in our definition) always has the same *dimensionality* as its host array, but its *effective dimensionality* may be lower.

Along any particular dimension, a rolling window can have size no larger than the size of its host array along that dimension, in which case we can say it fills the host array along that dimension. Applying a rolling window that completely fills its host array’s along any dimensions results in a series of higher-dimensional views of lower-dimensional subarrays. For example, had we taken a rolling window of size `(1, 9)` over arr, we would have gotten as a result<br><br>
<center><code>[[ 0,  1,  2,  3,  4,  5,  6,  7,  8]],
[[ 9, 10, 11, 12, 13, 14, 15, 16, 17]],
[[18, 19, 20, 21, 22, 23, 24, 25, 26]],
[[27, 28, 29, 30, 31, 32, 33, 34, 35]],
[[36, 37, 38, 39, 40, 41, 42, 43, 44]],
[[45, 46, 47, 48, 49, 50, 51, 52, 53]],
[[54, 55, 56, 57, 58, 59, 60, 61, 62]] 
</code></center>

Which is simply iterating over each row of `arr`, but taking a 2-d view of each 1-d row. To iterate over the columns of `arr`, we could apply a rolling window of size `(7, 1)`:

<center><code>[[ 0]   [[ 1]   [[ 2]   [[ 3]   [[ 4]   [[ 5]   [[ 6]   [[ 7]   [[ 8] 
 [ 9]    [10]    [11]    [12]    [13]    [14]    [15]    [16]    [17] 
 [18]    [19]    [20]    [21]    [22]    [23]    [24]    [25]    [26] 
 [27]    [28]    [29]    [30]    [31]    [32]    [33]    [34]    [35] 
 [36]    [37]    [38]    [39]    [40]    [41]    [42]    [43]    [44] 
 [45]    [46]    [47]    [48]    [49]    [50]    [51]    [52]    [53] 
 [54]],  [55]],  [56]],  [57]],  [58]],  [59]],  [60]],  [61]],  [62]]
</code></center>

In general, given an array `A` with `d` dimensions, a rolling window with size `1` along `n` of its dimensions (with `0 <= n <= d`) will be a `d`-dimensional view of a `(d-n)` dimensional subarray. If `n=0` (it doesn't have size 1 along any dimensions), the subarray’s effective dimensionality is the same as its dimensionality.

## A rolling window superarray: `skimage.view_as_windows`

There are different occasions when we would want to iterate over an array in sliding windows, applying some aggregating function to each slice; for example, applying a smoothing function or convolutional filter in image processing. If our sliding window has dimensions $$(w_0,\ w_1,\ …,\ w_{d-1})$$<br>on an array with dimensions $$(a_0,\ a_1,\ …,\ a_{d-1})$$<br>and the aggregating function yields a scalar output, then the output array will have dimensions $$(1 + a_0-w_0, 1 + a_1-w_1, …, 1 + a_{d-1}+w_{d-1})$$

The question is, how to implement a sliding window? A few considerations arise:
* Iterating over arrays of arbitrary dimensionality—we can’t explicitly write out for-loops of arbitrary depth;
* Keeping track of indices mapping from host array to output array—in principle not too difficult, but easy to have bugs in practice;
* Minimizing memory usage—we don’t want to copy subarrays as we iterate through them if we don’t have too, we just want to aggregate over them; and we certainly don’t want to copy all possible sliding windows simultaneously.

We could image using numpy’s view capabilities to take a higher-dimensional view into an array, enumerating all possible sliding windows with a given slice, and processing the results into an output array; but doing so from scratch would be somewhat tedious (not the worst, but you'd have to get the stride arithmetic right). Fortunately, the grunt work was already done for us: the [skimage](https://scikit-image.org/) package implements a function [`view_as_windows`](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=view_as_windows#skimage.util.view_as_windows) that does exactly what we want. In its most basic form, it accepts two arguments: a numpy `ndarray` (using a compatible type such as a list of lists will not work as of scikit-image 0.15.0), and a `window` (either an `iterable` spelling out the window size along each dimension, or an `int` to specify the size along all dimensions).

Let's say we call `view_as_windows` on an array `A` with dimensions $$(a_0,\ a_1,\ …,\ a_{d-1})$$<br>and a window `w` with dimensions$$(w_0,\ w_1,\ …,\ w_{d-1})$$<br>If we assign the results to a variable `view`, what properties does `view` have?
* The shape of `view` is the concatenation of two tuples:
    1. `1 + A.shape -  w` (here subtraction/addition are element-wise operations)
    2. `w`
* Indexing into `view` with some index $(i_0, i_1, …, i_{d-1})$ yields a view of `A` equivalent to the slice<br><center><code>A</code>[$i_0:i_0+w_0$<code>,</code>$i_1:i_1+w_1$<code>,</code>$i_{d-1}:i_{d-1}+w_{d-1}$<code>]</code></center><br>Recall that along any particular dimension `j`, we can start the sliding window at an index no more than $a_j-w_j$. This means indices in the range $[0,\ 1+a_j-w_j)$ are valid, which is the scope of `range(0, `$1+a_j-w_j$`)` in Python.

Returning to our sample problem from before, we can use the function as follows:

In [5]:
from skimage.util import view_as_windows

arr = np.arange(7*9).reshape(7, 9)
w = (2, 4)
arr_view = view_as_windows(arr, w)

print(arr.shape)
print(arr_view.shape)

(7, 9)
(6, 6, 2, 4)


Observe that the shapes of `arr` and `arr_view` obey the expected dimensionality relation:

In [6]:
np.array_equal(
    arr_view.shape,
    np.append(
        1+np.array(arr.shape)-w,
        w
    ) 
)

True

And indexing works as expected:

In [7]:
all(
    np.array_equal(
        arr_view[i, j],
        arr[i:i+w[0], j:j+w[1]]
    )
    for i in range(arr.shape[0]-w[0]+1)
        for j in range (arr.shape[1]-w[1]+1)
)

True

## Using `skimage.view_as_windows` to compute aggregating functions

Now the question is, how do we use the superarray generated by `view_as_window` to perform aggregation, e.g. a smoothing function such as a rolling mean?

Remember, the aggregating function acts over each slice of the original array traversed by the rolling window, in the same order in which the original array would be traversed over its individual values. So we want to be able to iterate through each slice and call the aggregating function on it.

`view_as_windows` gave us a way to iterate over each slice; we could manually create a new array with the expected dimensionality of the output, and then fill in the values one-by-one with for loops. This will work in many applications, where images are of low dimensions (2 or 3), but for higher-dimensional problems this may not be as scalable. Moreover, explicit for-loops in Python are not the most efficient way of performing simple operations over numpy arrays. Both scalability and efficiency are maintained if we use numpy's vectorized operations to do our work.

But how do we aggregate over the view returned by `view_as_window`? Remember, indexing into the view (with an index that is compatible with the original array) returns a slice into the original array of the given window size. This means that, given an original array with `d` dimensions, all of the slices in the view are contained in the latter `d` dimensions of the view, while the first `d` dimensions index over these slices. So, we call the aggregating function of interest over the last d dimensions of our array.

For the example we had above, `arr` has 2 dimensions. This means that the view into it has 4 dimensions, and that the slices of interest lie along the last 2 axes. So we can simply call a numpy aggregating function (ufunc) with the `axis` argument being the last two axes:

In [8]:
arr_view_mean = arr_view.mean(axis=(-2,-1))
arr_view_mean

array([[ 6.,  7.,  8.,  9., 10., 11.],
       [15., 16., 17., 18., 19., 20.],
       [24., 25., 26., 27., 28., 29.],
       [33., 34., 35., 36., 37., 38.],
       [42., 43., 44., 45., 46., 47.],
       [51., 52., 53., 54., 55., 56.]])

This is the same result we would have gotten had we done this manually:

In [9]:
arr_view_mean_manual = [
    [arr_view[i,j].mean()
     for j in range (arr.shape[1]-w[1]+1)]
    for i in range(arr.shape[0]-w[0]+1)
]

arr_mean_manual = [
    [arr[i:i+w[0],j:j+w[1]].mean()
     for j in range (arr.shape[1]-w[1]+1)]
    for i in range(arr.shape[0]-w[0]+1)
    
]

np.allclose(arr_view_mean_manual, arr_mean_manual) and \
    np.allclose(arr_mean_manual, arr_view_mean)

True

Other numpy ufuncs will also work here--for example, sum, prod, nansum, nanmean, etc.:

In [10]:
np.array_equal(
    arr_view.prod(axis=(-1,-2)),
    [
        [arr_view[i,j].prod()
         for j in range (arr.shape[1]-w[1]+1)]
        
        for i in range(arr.shape[0]-w[0]+1)
    ]
)

True

In addition, for convenience [SciPy](https://docs.scipy.org/doc/) provides us with the geometric mean and harmonic mean available from scipy.stats.mstats.gmean ([v1.6.0 source]()) and scipy.stats.hmean ([v1.6.0 source](https://github.com/scipy/scipy/blob/v1.6.0/scipy/stats/stats.py#L414-L479)):

In [11]:
from scipy.stats.mstats import gmean
from scipy.stats import hmean

# temporarily add 1 to arr to prevent the lowest value from being 0
# because the geometric/harmonic means don't work with 0
# since arr_view is a *view* into arr, it sees the incremented values
arr+=1


# Note: at the spot indicated below, the axis=(0,1) argument is required
# because gmean does not automatically flatten the array in the absence
# of an explicit axis argument, unlike numpy, which flattens it by default
print(np.allclose(
    gmean(arr_view, axis=(-1,-2)),
    [
        [gmean(arr_view[i,j], axis=(0,1)) # this is where the above comment applies
         for j in range (arr.shape[1]-w[1]+1)]
        
        for i in range(arr.shape[0]-w[0]+1)
    ]
))

# hmean does not support passing a tuple for the axis value
# so we have to do a little more work here
v0,v1,v2,v3 = arr_view.shape
print(np.allclose(
    hmean(arr_view.reshape(v0, v1, v2*v3),axis=-1) # reshapes the array *without* making a copy
         .reshape(arr.shape[0]-w[0]+1, arr.shape[1]-w[1]+1), # it works, but wow is this clunky
    [
        [hmean(arr_view[i,j].ravel())
         for j in range (arr.shape[1]-w[1]+1)]
        
        for i in range(arr.shape[0]-w[0]+1)
    ]
))
arr-=1 # don't forget to restore

True
True


The gmean solution `gmean(arr_view, axis=(-1,-2))` is as succinct as the pure numpy ufuncs. However, as noted, the hmean solution is rather clunky. Why don't we define one ourselves? It is easy to do so by building on top of numpy's vectorized operations, and if you look at the source links provided, this is exactly what SciPy does; the hmean implementation just stops short of allowing a tuple for the `axis` argument.

Here's how we do it: for a given set of numbers ${x_i}$ the harmonic mean is defined as (mouthful) the reciprocal of the average of the reciprocals, or in notation:
$$h\ =\ \left(\frac{1}{n}\ \sum_{i=1}^{n}{\frac{1}{x_i}}\right)^{-1}\ =\ n\ /\ \sum_{i=1}^{n}{\frac{1}{x_i}}$$

Given numpy's vectorization opeations, this is easy to accomplish. The somewhat tricky part is making sure we get the correct value of `n` when we are dealing with multiple axes; however, numpy again provides us the convenient `prod` function to accomplish this with a single function call for a given `array`: `n = np.prod(array.shape)`.

Given this, our hmean function might look as follows:

```python
def hmean(array, axis):
    if np.any(array < 0):
        raise ValueError("cannot compute harmonic mean on dataset containing 0")
    axis = np.atleast_1d(axis) # ensures that `axis` is iterable
    return np.prod([array.shape[i] for i in axis]) / np.reciprocal(array).sum(axis)
```

We call <code>np.prod</code> on the dimensions along the axes where the array is being reduced to get the total number of elements contained in a reduction along these axes.

Although this is a fully functional `hmean` implementation, there is a problem with using it on the output of `view_as_windows`. As cautioned [in the documentation for this function](https://scikit-image.org/docs/dev/api/skimage.util.html#skimage.util.view_as_windows), the function returns a *view* into the original array, which ensures that space is not allocated for an array which has the size of the view. However, the moment you operate on the view with a *non-reductional function* (so as to create another array of the same size), memory must be allocated to match the size of the view, which has the curse of dimensionality (extra memory allocated grows *fast* with dimensionality). In the above `hmean` function, there are two calls that cause this problem: `array < 0`, and `np.reciprocal`. Unfortuantely, without digging into the C API for Python & NumPy, the only other solution is to write a custom function that computes a rolling harmonic mean by acting on the original array rather than the view:
```python
def rolling_hmean(array, window, axis, _raise0 = True):
    if _raise0 and np.any(array < 0):
        raise ValueError("cannot compute harmonic mean on dataset containing 0")
    
    r = np.reciprocal(array) # if 0's are present, contains nan results
    v = view_as_windows(array, window)
    
    axis = np.atleast_1d(axis) # ensures that `axis` is iterable
    # the dimensions in the view to aggregate over must be the latter half
    return np.prod([array.shape[i] for i in axis]) / v.sum(array.ndim + axis)
```

This is actually not so bad--let's test it.

In [12]:
def rolling_hmean(array, window, axis=None, _raise0 = True):
    if _raise0 and np.any(array < 0):
        raise ValueError("cannot compute harmonic mean on dataset containing 0")

    r = np.reciprocal(array.astype(float)) # if 0's are present, contains nan results
    v = view_as_windows(r, window)

    if axis is None: # all axes
        axis = np.arange(array.ndim, dtype=int)
    else:
        axis = np.atleast_1d(axis) # ensures that `axis` is iterable
    # the dimensions in the view to aggregate over must be the latter half
    return np.prod([w[i] for i in axis]) / v.sum(tuple(array.ndim + axis))

arr+=1
print(np.allclose(
    # clunky solution from before
    hmean(arr_view.reshape(v0, v1, v2*v3),axis=-1)
         .reshape(arr.shape[0]-w[0]+1, arr.shape[1]-w[1]+1),
    
    rolling_hmean(arr, w) # nice and simple
))

arr-=1 # don't forget to restore

True


We have thus regained our convenient (& *scalable* + *efficient*) syntax.

While we're at it, why don't we write convenience functions for the sum and mean?

In [13]:
def rolling_mean(array, window):
    return view_as_windows(array, window).mean(
        axis=tuple(array.ndim+d for d in range(array.ndim))
    )

def rolling_sum(array, window):
    return view_as_windows(array, window).sum(
        axis=tuple(array.ndim+d for d in range(array.ndim))
    )

print(np.allclose(arr_view.mean(axis=(-2,-1)), rolling_mean(arr, w)))
print(np.array_equal(arr_view.sum(axis=(-2,-1)), rolling_sum(arr, w)))

True
True


## Extension: weighted mean & sum

Taking weighted means is very common, but the above functions don't do this for us. Fortunately, because `np.average` provides an inbuilt `weights` argument, it is simple to add this functionality to our custom `rolling_mean`:

In [14]:
def rolling_mean(array, window, weights=None):
    if weights is None:
        return view_as_windows(array, window).mean(
            axis=tuple(array.ndim+d for d in range(array.ndim))
        )
    
    v = view_as_windows(array, window)
    weights = np.asarray(weights) # ensures a `shape` attribute

    if not np.array_equal(weights.shape, window):
        raise ValueError(
            "`weights` must have shape equivalent to `window` ("
            f"weights has shape {weights.shape}, but window is {window})"
        )
    
    return np.average(
        v,
        axis=tuple(array.ndim+d for d in range(array.ndim)),
        weights = np.broadcast_to(weights, v.shape)
    )

print(np.allclose(
    rolling_mean(arr, w, weights=np.full(w,1)),
    rolling_mean(arr, w) # calling with no weights == calling with unitary weights
))

print(np.allclose(
    rolling_mean(arr, w, weights=[[.1,.2,.3,.4],[.4,.3,.2,.1]]),
    
    [
        [np.average(arr_view[i,j], weights=[[.1,.2,.3,.4],[.4,.3,.2,.1]])
         for j in range (arr.shape[1]-w[1]+1)]
        for i in range(arr.shape[0]-w[0]+1)
    ]
))

True
True


What about a weighted sum? Numpy does not offer a weight argument on the `sum` ufunc, and we would rather not have to copy intermediate slices of the output of `view_as_windows` (or the entire view array). Fortunately, a weighted sum can be expressed in terms of the weighted mean. Given a set of weights $W$ and data $X$, the weighted mean is $$\overline{X_W}\ =\ \frac{\sum{\left(W\bullet X\right)}}{\sum{W}}$$<br>
Where the numerator, a dot product, is the weighted sum we seek. Simply rearrange terms:
$$\sum{\left(W\bullet X\right)}\ =\ \overline{X_W} \sum{W}$$<br>
And we have the answer.

This means that, given a way of computing a rolling weighted mean, all we have to do is reverse-engineer it by multiplying the result by the sum of the weights. A sample implementation is
```python
def rolling_sum(array, window, weights=None):
    if weights is None:
        return view_as_windows(array, window).sum(
            axis=tuple(array.ndim+d for d in range(array.ndim))
        )
    
    weights = np.asarray(weights)

    if not np.array_equal(weights.shape, window):
        raise ValueError(
            "`weights` must have shape equivalent to `window` ("
            f"weights has shape {weights.shape}, but window is {window})"
        )
    
    # weighted sum = weighted mean * sum of weights
    return np.average(
        v,
        axis=tuple(array.ndim+d for d in range(array.ndim)),
        weights = np.broadcast_to(weights,v.shape)
    ) * weights.sum()
```
Let's give that a try.

In [15]:
def rolling_sum(array, window, weights=None):
    if weights is None:
        return view_as_windows(array, window).sum(
            axis=tuple(array.ndim+d for d in range(array.ndim))
        )

    weights = np.asarray(weights) # ensures a `shape` attribute

    if not np.array_equal(weights.shape, window):
        raise ValueError(
            "`weights` must have shape equivalent to `window` ("
            f"weights has shape {weights.shape}, but window is {window})"
        )

    v = view_as_windows(array, window)
    # weighted sum = weighted mean * sum of weights
    return np.average(
        v,
        axis=tuple(array.ndim+d for d in range(array.ndim)),
        weights = np.broadcast_to(weights,v.shape)
    ) * weights.sum()

print(np.allclose(
    rolling_sum(arr, w, np.full(w,1)),
    rolling_sum(arr, w)
))

print(np.allclose(
    rolling_sum(arr, w, weights=[[1,2,3,4],[4,3,2,1]]),
    
    [ # weights sum to 20
        [20*np.average(arr_view[i,j], weights=[[1,2,3,4],[4,3,2,1]])
         for j in range (arr.shape[1]-w[1]+1)]
        for i in range(arr.shape[0]-w[0]+1)
    ]
))

True
True


Of course, it would be better if `np.sum` implemented a `weights` argument, thus avoid the extra work involved in creating an intermediate array in the rolling_sum function, but without digging into the C API, this is about the best we can do with a simple, pure-Python implementation. Note, we could also multiply the weighted-mean array by the sum of the weights in-place, but you would have to make sure that the dtypes matched, otherwise numpy would raise an error. This is not too difficult in practice. In principle, with this logic one could also implement weighted geometric/harmonic means without too much difficulty, as well as weighted products. These are left as exercises for the reader.

## The takeaways

Using **vectorized** operations in numpy is strongly advantageous to explicit loops in Python for two reasons: simpler code, and greater efficiency.

Numpy's **view semantics** allow us to loop and aggregate over subarrays of arbitrary dimensionality efficiently (without memory or time overhead involved in copying data).

**Scikit-image** provides the `view_as_windows` function, which allows us to iterate and aggregate over a numpy array using a rolling window in a way that capitalizes on both vectorized operations and view semantics.

Most important: before coding your own solution for a problem, see if somebody else has done a lot of the gruntwork for you. In this case, numpy and scikit-image greatly simplified a problem that would have been much more difficult to code from scratch.